In [15]:
import numpy as np
a = np.load('/home/user/ckwan1/ml/mlsimdata_npy/mlsimdata1/213864.npy')

In [18]:
a[:,:,:,6:9].shape

(32, 32, 32, 3)

In [12]:
import torch
import torch.nn as nn
import torch.optim as optim
import lightning.pytorch as pl
from periodic_padding import periodic_padding_3d

def crop_tensor(x):
	x = x.narrow(2,1,x.shape[2]-3).narrow(3,1,x.shape[3]-3).narrow(4,1,x.shape[4]-3).contiguous()
	return x

def conv3x3(inplane,outplane, stride=1,padding=0):
	return nn.Conv3d(inplane,outplane,kernel_size=3,stride=stride,padding=padding,bias=True)
    
# Assuming conv3x3 and BasicBlock are defined as in your original code.
class BasicBlock(nn.Module):
	def __init__(self,inplane,outplane,stride = 1):
		super(BasicBlock, self).__init__()
		self.conv1 = conv3x3(inplane,outplane,padding=0,stride=stride)
		self.bn1 = nn.BatchNorm3d(outplane)
		self.relu = nn.ReLU(inplace=True)

	def forward(self,x):
		x = periodic_padding_3d(x,pad=(1,1,1,1,1,1))
		out = self.conv1(x)
		out = self.bn1(out)
		out = self.relu(out)
		return out


class Lpt2NbodyNet(nn.Module):
    def __init__(self, block):
        super(Lpt2NbodyNet, self).__init__()
        self.layer1 = self._make_layer(block, 3, 64, blocks=2, stride=1)
        self.layer2 = self._make_layer(block, 64, 128, blocks=1, stride=2)
        self.layer3 = self._make_layer(block, 128, 128, blocks=2, stride=1)
        self.layer4 = self._make_layer(block, 128, 256, blocks=1, stride=2)
        self.layer5 = self._make_layer(block, 256, 256, blocks=2, stride=1)
        self.deconv1 = nn.ConvTranspose3d(256, 128, 3, stride=2, padding=0)
        self.deconv_batchnorm1 = nn.BatchNorm3d(num_features=128, momentum=0.1)
        self.layer6 = self._make_layer(block, 256, 128, blocks=2, stride=1)
        self.deconv2 = nn.ConvTranspose3d(128, 64, 3, stride=2, padding=0)
        self.deconv_batchnorm2 = nn.BatchNorm3d(num_features=64, momentum=0.1)
        self.layer7 = self._make_layer(block, 128, 64, blocks=2, stride=1)
        self.deconv4 = nn.ConvTranspose3d(64, 3, 1, stride=1, padding=0)

    def _make_layer(self, block, inplanes, outplanes, blocks, stride=1):
        layers = []
        for _ in range(blocks):
            layers.append(block(inplanes, outplanes, stride=stride))
            inplanes = outplanes
        return nn.Sequential(*layers)

    def forward(self, x):
        x1 = self.layer1(x)
        x = self.layer2(x1)
        x2 = self.layer3(x)
        x = self.layer4(x2)
        x = self.layer5(x)
        x = periodic_padding_3d(x, pad=(0, 1, 0, 1, 0, 1))
        x = nn.functional.relu(self.deconv_batchnorm1(crop_tensor(self.deconv1(x))), inplace=True)
        x = torch.cat((x, x2), dim=1)
        x = self.layer6(x)
        x = periodic_padding_3d(x, pad=(0, 1, 0, 1, 0, 1))
        x = nn.functional.relu(self.deconv_batchnorm2(crop_tensor(self.deconv2(x))), inplace=True)
        x = torch.cat((x, x1), dim=1)
        x = self.layer7(x)
        x = self.deconv4(x)
        return x

class UNet3D(nn.Module):  
    def __init__(self, block, num_layers=2, base_filters=64, blocks_per_layer=2):
        super(UNet3D, self).__init__()
        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        
        # Encoder path
        init_channels = 3
        out_channels = base_filters
        self.init_conv = self._make_layer(block, init_channels, out_channels, blocks=blocks_per_layer, stride=1)
        for _ in range(num_layers):
            self.encoders.append(self._make_layer(block, out_channels, out_channels*2, blocks=1, stride=2))
            self.encoders.append(self._make_layer(block, out_channels*2, out_channels*2, blocks=blocks_per_layer, stride=1))
            out_channels *= 2

        # Decoder path
        for _ in range(num_layers):
            self.decoders.append(nn.ConvTranspose3d(out_channels, out_channels//2, kernel_size=3, stride=2, padding=0))
            self.decoders.append(self._make_layer(block, out_channels, out_channels//2, blocks=blocks_per_layer, stride=1))
            out_channels //= 2

        self.final_conv = nn.ConvTranspose3d(out_channels, 3, 1, stride=1, padding=0)

        # Predefine BatchNorm3d and ReLU layers for each decoder step
        self.batch_norms = nn.ModuleList()
        self.relu = nn.ReLU(inplace=True)
        for i in range(num_layers):
            self.batch_norms.insert(0, nn.BatchNorm3d(base_filters * (2 ** i)))  # Adjust channels accordingly

    def _make_layer(self, block, inplanes, outplanes, blocks, stride=1):
        layers = []
        for _ in range(blocks):
            layers.append(block(inplanes, outplanes, stride=stride))
            inplanes = outplanes
        return nn.Sequential(*layers)

    def forward(self, x):
        encoder_outputs = []

        x = self.init_conv(x)
        encoder_outputs.append(x)
        
        # Encoding path
        for i in range(0, len(self.encoders), 2):
            x = self.encoders[i](x)  # Compression layer
            x = self.encoders[i + 1](x)  # Non-compression layer
            encoder_outputs.append(x)

        # Decoding path
        for i in range(0, len(self.decoders), 2):
            x = periodic_padding_3d(x, pad=(0, 1, 0, 1, 0, 1))  # Assuming this is a custom function
            x = self.decoders[i](x)  # Transpose Conv layer
            x = crop_tensor(x)  # Assuming this is a custom function to crop the tensor
            
            # Use the pre-defined BatchNorm3d and ReLU layers
            x = self.batch_norms[i // 2](x)  # BatchNorm
            x = nn.ReLU(inplace=True)(x)  # ReLU
            
            # Skip connection with encoder outputs
            x = torch.cat((x, encoder_outputs[len(encoder_outputs)-2-i//2]), dim=1)  # Skip connection
            
            x = self.decoders[i + 1](x)  # Non-compression layer

        # Final 1x1 Conv
        x = self.final_conv(x)
        return x

In [13]:
import torch
import torch.nn as nn

class UNet3D(nn.Module):
    def __init__(self, block, num_layers=2, base_channels=64):
        super(UNet3D, self).__init__()
        self.encoders = nn.ModuleList()
        self.decoders = nn.ModuleList()
        
        # Encoder path
        init_channels = 3
        out_channels = base_channels
        self.init_conv = self._make_layer(block, init_channels, out_channels, blocks=2, stride=1)
        print(f"Initial conv output channels: {out_channels}")

        for _ in range(num_layers):
            self.encoders.append(self._make_layer(block, out_channels, out_channels*2, blocks=1, stride=2))
            self.encoders.append(self._make_layer(block, out_channels*2, out_channels*2, blocks=2, stride=1))
            out_channels *= 2
        
        # Decoder path
        for _ in range(num_layers):
            self.decoders.append(nn.ConvTranspose3d(out_channels, out_channels//2, kernel_size=3, stride=2, padding=0))
            self.decoders.append(self._make_layer(block, out_channels, out_channels//2, blocks=2, stride=1))
            out_channels //= 2

        self.final_conv = nn.ConvTranspose3d(out_channels, 3, kernel_size=1, stride=1, padding=0)

    def _make_layer(self, block, inplanes, outplanes, blocks, stride=1):
        layers = []
        for _ in range(blocks):
            layers.append(block(inplanes, outplanes, stride=stride))
            inplanes = outplanes
        return nn.Sequential(*layers)

    def forward(self, x):
        encoder_outputs = []

        x = self.init_conv(x)
        print(f"After initial conv: {x.shape}")
        encoder_outputs.append(x)

        # Encoding path
        for i in range(0, len(self.encoders), 2):
            x = self.encoders[i](x)  # Compression layer
            print(f"After encoder compression {i//2}: {x.shape}")
            x = self.encoders[i + 1](x)  # Non-compression layer
            print(f"After encoder non-compression {i//2}: {x.shape}")
            encoder_outputs.append(x)

        # Decoding path
        for i in range(0, len(self.decoders), 2):
            x = periodic_padding_3d(x, pad=(0, 1, 0, 1, 0, 1))
            print(f"After periodic padding before transpose conv {i//2}: {x.shape}")
            x = self.decoders[i](x)  # Transpose Conv layer
            print(f"After transpose conv {i//2}: {x.shape}")
            x = crop_tensor(x)  # Crop to match dimensions if necessary
            print(f"After cropping {i//2}: {x.shape}")
            
            # BatchNorm and ReLU before concatenation
            x = nn.BatchNorm3d(x.shape[1])(x)  # BatchNorm
            x = nn.ReLU(inplace=True)(x)  # ReLU
            print(f"After BatchNorm and ReLU {i//2}: {x.shape}")

            x = torch.cat((x, encoder_outputs[len(encoder_outputs)-2-i//2]), dim=1)  # Skip connection
            print(f"After concatenation {i//2}: {x.shape}")
            
            x = self.decoders[i + 1](x)  # Non-compression layer
            print(f"After decoder non-compression {i//2}: {x.shape}")

        # Final 1x1 Conv
        x = self.final_conv(x)
        print(f"Final output shape: {x.shape}")
        return x


In [14]:
model = UNet3D(block=BasicBlock, num_layers=3, base_channels=64)

Initial conv output channels: 64


In [12]:
model = UNet3D(block=BasicBlock, num_layers=3, base_filters=64)

In [15]:
input_shape = (1, 3, 32, 32, 32)  # Change the dimensions as needed
input_tensor = torch.rand(input_shape)  # Create a random input tensor

# Run a forward pass through the model
with torch.no_grad():  # Disable gradient calculation for inference
    output = model(input_tensor)

After initial conv: torch.Size([1, 64, 32, 32, 32])
After encoder compression 0: torch.Size([1, 128, 16, 16, 16])
After encoder non-compression 0: torch.Size([1, 128, 16, 16, 16])
After encoder compression 1: torch.Size([1, 256, 8, 8, 8])
After encoder non-compression 1: torch.Size([1, 256, 8, 8, 8])
After encoder compression 2: torch.Size([1, 512, 4, 4, 4])
After encoder non-compression 2: torch.Size([1, 512, 4, 4, 4])
After periodic padding before transpose conv 0: torch.Size([1, 512, 5, 5, 5])
After transpose conv 0: torch.Size([1, 256, 11, 11, 11])
After cropping 0: torch.Size([1, 256, 8, 8, 8])
After BatchNorm and ReLU 0: torch.Size([1, 256, 8, 8, 8])
After concatenation 0: torch.Size([1, 512, 8, 8, 8])
After decoder non-compression 0: torch.Size([1, 256, 8, 8, 8])
After periodic padding before transpose conv 1: torch.Size([1, 256, 9, 9, 9])
After transpose conv 1: torch.Size([1, 128, 19, 19, 19])
After cropping 1: torch.Size([1, 128, 16, 16, 16])
After BatchNorm and ReLU 1: torch

In [14]:
model

UNet3D(
  (encoders): ModuleList(
    (0): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2))
        (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
    )
    (1): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
        (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
      (1): BasicBlock(
        (conv1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
        (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
      )
    )
    (2): Sequential(
      (0): BasicBlock(
        (conv1): Conv3d(128, 256, kernel_size=(3, 3, 3), stride=(2, 2, 2))
        (bn1): BatchNorm3d(256, eps=1e-05, momentum=0.1, affine=True, track_running

In [ ]:
%debug 


> /tmp/ipykernel_62867/2134185488.py(120)forward()
    118             x = nn.ReLU(inplace=True)(x)  # ReLU
    119 
--> 120             x = torch.cat((x, encoder_outputs[-(i + 1)]), dim=1)  # Skip connection
    121 
    122             x = self.decoders[2 * i + 1](x)  # Non-compression layer

*** Newest frame
*** Newest frame


In [5]:
model = Lpt2NbodyNet(block=BasicBlock)
print(model)

Lpt2NbodyNet(
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(3, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1): BasicBlock(
      (conv1): Conv3d(64, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      (bn1): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
  )
  (layer2): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(64, 128, kernel_size=(3, 3, 3), stride=(2, 2, 2))
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
  )
  (layer3): Sequential(
    (0): BasicBlock(
      (conv1): Conv3d(128, 128, kernel_size=(3, 3, 3), stride=(1, 1, 1))
      (bn1): BatchNorm3d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
    )
    (1

In [2]:
import numpy as np

def swap(array, i, j):
    array[[i, j]] = array[[j, i]]
    return array

data = np.load('data_collection/mlsimdata_npy2/mlsimdata1/5995944.npy')
data = np.einsum('ijkl->lijk', data)
LPT = data[6:9,:,:,:]  
Nbody = data[0:3,:,:,:]  
for i in range(1000):
    if np.random.rand() < .5:
        LPT = LPT[:, ::-1, ...]
        LPT[0] = -LPT[0]
        Nbody = Nbody[:, ::-1, ...]
        Nbody[0] = -Nbody[0]
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    if np.random.rand() < .5:
        LPT = LPT[:, :, ::-1, ...]
        LPT[1] = -LPT[1]
        Nbody = Nbody[:, :, ::-1, ...]
        Nbody[1] = -Nbody[1]
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    if np.random.rand() < .5:
        LPT = LPT[:, :, :, ::-1]
        LPT[2] = -LPT[2]
        Nbody = Nbody[:, :, :, ::-1]
        Nbody[2] = -Nbody[2]
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    prand = np.random.rand()
    if prand < 1./6:
        LPT = np.transpose(LPT, axes=(0, 2, 3, 1))
        LPT = swap(LPT, 0, 2)
        LPT = swap(LPT, 0, 1)
        Nbody = np.transpose(Nbody, axes=(0, 2, 3, 1))
        Nbody = swap(Nbody, 0, 2)
        Nbody = swap(Nbody, 0, 1)
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    elif prand < 2./6:
        LPT = np.transpose(LPT, axes=(0, 2, 1, 3))
        LPT = swap(LPT, 0, 1)
        Nbody = np.transpose(Nbody, axes=(0, 2, 1, 3))
        Nbody = swap(Nbody, 0, 1)
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    elif prand < 3./6:
        LPT = np.transpose(LPT, axes=(0, 1, 3, 2))
        LPT = swap(LPT, 1, 2)
        Nbody = np.transpose(Nbody, axes=(0, 1, 3, 2))
        Nbody = swap(Nbody, 1, 2)
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    elif prand < 4./6:
        LPT = np.transpose(LPT, axes=(0, 3, 1, 2))
        LPT = swap(LPT, 1, 2)
        LPT = swap(LPT, 0, 1)
        Nbody = np.transpose(Nbody, axes=(0, 3, 1, 2))
        Nbody = swap(Nbody, 1, 2)
        Nbody = swap(Nbody, 0, 1)
        print(np.any(LPT>128))
        print(np.any(Nbody>128))
    elif prand < 5./6:
        LPT = np.transpose(LPT, axes=(0, 3, 2, 1))
        LPT = swap(LPT, 0, 2)
        Nbody = np.transpose(Nbody, axes=(0, 3, 2, 1))
        Nbody = swap(Nbody, 0, 2)
        print(np.any(LPT>128))
        print(np.any(Nbody>128))


False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [1]:
from datapile import HuggingfaceLoader
data = HuggingfaceLoader(
    dataset_path="/home/user/ckwan1/ml/huggingface_dataset", 
    batch_size=1, 
    shuffle=False,
    num_workers=1,
)
data.setup("validation")
val_iter = iter(data.val_dataloader())
batch = next(val_iter)


/home/user/ckwan1/.conda/envs/d3m/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
x,y=batch

In [3]:
y.shape

torch.Size([1, 4, 32, 32, 32])

In [4]:
val_iter

In [ ]:
import numpy as np
d = np.load('/home/user/ckwan1/ml/fastpm-master/mlsim/data_collection/mlsimdata_npy2_density/mlsimdata1/213864.npy')
print(np.expand_dims(d[0], axis=0).shape)

(1, 32, 32, 32)


In [10]:
d1 = np.load('/home/user/ckwan1/ml/data_collection/mlsimdata_npy2/mlsimdata1/213864.npy')
d1 = np.einsum('ijkl->lijk', d1)
print(d1[0:3].shape)

(3, 32, 32, 32)


In [15]:
np.vstack((np.expand_dims(d[0], axis=0),d1[0:3])).shape

(4, 32, 32, 32)

In [5]:
from denoising_diffusion_3D import Trainer3D, Unet3D, GaussianDiffusion3D
import torch

model = Unet3D(
    dim = 64,
    dim_mults = (1, 2, 4, 8)
)
diffusion = GaussianDiffusion3D(
    model,
    image_size = 16,
    timesteps = 1000    # number of steps
)

In [11]:
training_images = torch.randn(1, 3, 16, 16, 16) # images are normalized from 0 to 1
loss = diffusion.model_predictions(training_images,0)



AttributeError: 'int' object has no attribute 'device'

In [1]:
import os
import pandas as pd
import numpy as np
import torch
from datasets import Dataset

def load_numpy_files_generator(shards):
    # This generator takes a list of file paths (a shard) and yields examples.
    for file_path in shards:
        print(file_path)
        # Load the numpy array and convert to a torch tensor.
        displacement_array = np.load(file_path)
        displacement_tensor = torch.from_numpy(displacement_array)
        
        # Extract file id from the file name.
        file_name = os.path.basename(file_path)
        id = os.path.splitext(file_name)[0]
        
        yield {
            'displacement': displacement_tensor.astype(np.float32),
            'id': id,
        }

def main():
    csv_file_displacement = '/home/user/ckwan1/ml/displacement.csv'
    save_path = '/home/user/ckwan1/ml/huggingface_dataset_128_mu/'

    if not os.path.exists(save_path):
        os.makedirs(save_path)
    
    # Read CSV to get the complete list of file paths.
    file_paths = pd.read_csv(csv_file_displacement)['file_path'].tolist()

    # Define the number of processes you'd like to use.
    num_proc = 32
    
    # Split file_paths into num_proc shards.
    # For example, if file_paths = [a, b, c, d, e, f, ...],
    # then shards will be: [[a, e, ...], [b, f, ...], [c, g, ...], [d, h, ...]]
    shards = [file_paths[i::num_proc] for i in range(num_proc)]
    
    # Use from_generator with gen_kwargs set to our shards, and set num_proc.
    dataset = Dataset.from_generator(
        load_numpy_files_generator,
        gen_kwargs={"shards": shards},
        num_proc=num_proc
    )
    
    dataset.save_to_disk(save_path)
    print("Dataset saved successfully.")

if __name__ == "__main__":
    main()


/home/user/ckwan1/.conda/envs/d3m/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Generating train split: 0 examples [00:00, ? examples/s]

['mlsimdata_npy2_128/128ncmlsimdata1/2175000.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/292623607.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/34966635.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/54179685.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/117511130.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/6365520.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/869653956.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/650696148.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/383973296.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/221538570.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/107269776.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/337958481.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/414112080.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/129247580.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/15072073.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/664474528.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/10589202.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/205115136.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/358106424.npy', 'mlsimdata_npy2_128

Generating train split: 0 examples [00:00, ? examples/s]

['mlsimdata_npy2_128/128ncmlsimdata1/173954718.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/602239114.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/85340710.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/14468168.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/534513420.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/521994911.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/325049869.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/87667398.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/315058740.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/243882938.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/93458448.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/241404408.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/206799369.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/145108368.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/827085504.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/47880168.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/837876393.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/491331888.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/75083055.npy', 'mlsimdata_npy2_1

Generating train split: 0 examples [00:00, ? examples/s]

...

['mlsimdata_npy2_128/128ncmlsimdata1/361193058.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/510669777.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/47496890.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/373430875.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/177551140.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/208750554.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/120693232.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/678520024.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/70557333.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/38409870.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/383763552.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/58172842.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/628126105.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/292921650.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/566770473.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/56014848.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/135164480.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/77504101.npy', 'mlsimdata_npy2_128/128ncmlsimdata4/456184598.npy', 'mlsimdata_npy2_1

Generating train split: 0 examples [00:01, ? examples/s]


['mlsimdata_npy2_128/128ncmlsimdata1/228356550.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/136181574.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/36558060.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/238888158.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/737313300.npy', 'mlsimdata_npy2_128/128ncmlsimdata1/24142725.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/60512914.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/179527576.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/83514816.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/87782210.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/477807768.npy', 'mlsimdata_npy2_128/128ncmlsimdata2/450195680.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/155099945.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/163586304.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/383022510.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/197262898.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/569444456.npy', 'mlsimdata_npy2_128/128ncmlsimdata3/654176156.npy', 'mlsimdata_npy2_128/128ncmlsimdata4/47251031.npy', 'mlsimdata_npy2_

Generating train split: 0 examples [00:01, ? examples/s]


DatasetGenerationError: An error occurred while generating the dataset